# Lab 4: Task Dependencies and Branching - Managing Workflow Logic

## 🎯 Objectives
- Understand how to define task dependencies with bitshift operators
- Use BranchPythonOperator for conditional logic
- Apply trigger rules to handle failures
- Create dynamic tasks with task mapping
- Use TaskGroups to organize tasks
- Build complex conditional workflows

## 📋 Prerequisites
- Completed Lab 1-3
- Understand operators and tasks
- Airflow cluster is running

## 🏗️ Dependencies Overview
Task dependencies in Airflow can be defined using:
- **Bitshift operators**: `>>` (set downstream), `<<` (set upstream)
- **Methods**: `set_downstream()`, `set_upstream()`
- **Lists**: Multiple dependencies at once


## 1. Import Libraries and Setup


In [ ]:
# Import Airflow dependencies and branching
from airflow.sdk import DAG, task
from airflow.providers.standard.operators.empty import EmptyOperator
from airflow.providers.standard.operators.python import BranchPythonOperator, PythonOperator
from airflow.providers.standard.operators.bash import BashOperator
from airflow.sdk.task_group import TaskGroup

import pendulum
from datetime import datetime, timedelta
import random

print("✅ Airflow dependencies và branching modules imported successfully!")


## 2. Bitshift Operators - Defining Dependencies

Airflow uses bitshift operators (`>>` and `<<`) to define dependencies in a visual and Pythonic way.


In [ ]:
# DAG with bitshift operators
@dag(
    dag_id="bitshift_dependencies_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["dependencies", "bitshift"],
)
def bitshift_dependencies_dag():
    """
    ### Bitshift Operators Example
    Sample DAG using bitshift operators to define dependencies.
    """
    
    # Create tasks
    start = EmptyOperator(task_id="start")
    task_a = EmptyOperator(task_id="task_a")
    task_b = EmptyOperator(task_id="task_b")
    task_c = EmptyOperator(task_id="task_c")
    task_d = EmptyOperator(task_id="task_d")
    end = EmptyOperator(task_id="end")
    
    # Approach 1: Single dependency with >>
    # start >> task_a means task_a depends on start
    start >> task_a
    
    # Approach 2: Multiple downstream tasks
    # task_a runs before task_b and task_c
    task_a >> [task_b, task_c]
    
    # Approach 3: Multiple upstream tasks
    # task_d depends on both task_b and task_c
    [task_b, task_c] >> task_d
    
    # Approach 4: Chain dependencies
    task_d >> end
    
    # Or write more concisely:
    # start >> task_a >> [task_b, task_c] >> task_d >> end

# Create DAG
bitshift_dag = bitshift_dependencies_dag()

print("✅ Bitshift Dependencies DAG created!")
print(f"Tasks: {[task.task_id for task in bitshift_dag.tasks]}")
print("\n📊 Dependency Flow:")
print("start → task_a → [task_b, task_c] → task_d → end")


## 3. BranchPythonOperator - Conditional Branching

BranchPythonOperator allows selecting which branch will run based on conditions. Function must return task_id(s) of the next task(s).


In [ ]:
# DAG with BranchPythonOperator
@dag(
    dag_id="branching_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["branching", "conditional"],
)
def branching_dag():
    """
    ### Branching Example
    Sample DAG using BranchPythonOperator for conditional logic.
    """
    
    start = EmptyOperator(task_id="start")
    
    # Branch function - return task_id(s) of task(s) that will run
    def choose_branch(**context):
        """Choose branch based on condition"""
        execution_date = context['data_interval_start']
        day_of_week = execution_date.weekday()
        
        # Monday (0) or Friday (4): run both branches
        if day_of_week == 0 or day_of_week == 4:
            return ["branch_a", "branch_b"]
        # Weekend: only run branch_a
        elif day_of_week >= 5:
            return "branch_a"
        # Weekday: only run branch_b
        else:
            return "branch_b"
    
    branching = BranchPythonOperator(
        task_id="branching",
        python_callable=choose_branch,
    )
    
    # Branch tasks
    branch_a = EmptyOperator(task_id="branch_a")
    branch_b = EmptyOperator(task_id="branch_b")
    
    # Join task - runs after branches complete
    join = EmptyOperator(
        task_id="join",
        trigger_rule="none_failed_min_one_success",  # Run if at least 1 branch succeeds
    )
    
    end = EmptyOperator(task_id="end")
    
    # Define dependencies
    start >> branching
    branching >> branch_a >> join
    branching >> branch_b >> join
    join >> end

# Create DAG
branching_dag_instance = branching_dag()

print("✅ Branching DAG created!")
print(f"Tasks: {[task.task_id for task in branching_dag_instance.tasks]}")
print("\n💡 Branch logic:")
print("  - Monday/Friday: Run both branches")
print("  - Weekend: Run branch_a only")
print("  - Weekday: Run branch_b only")


## 4. Trigger Rules - Handling Task Failures

Trigger rules determine when a task will run based on the status of upstream tasks. Common trigger rules:
- `all_success` (default): All upstream tasks must succeed
- `all_failed`: All upstream tasks must fail
- `all_done`: Run when all upstream tasks complete (regardless of status)
- `one_failed`: Run when at least 1 upstream task failed
- `one_success`: Run when at least 1 upstream task succeeds
- `none_failed`: Run when no upstream tasks failed (success or skipped)
- `none_failed_min_one_success`: Run when no failed and at least 1 success
- `none_skipped`: Run when no upstream tasks are skipped


In [ ]:
# DAG with Trigger Rules
@dag(
    dag_id="trigger_rules_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["trigger-rules", "error-handling"],
)
def trigger_rules_dag():
    """
    ### Trigger Rules Example
    Sample DAG demonstrating different trigger rules.
    """
    
    start = EmptyOperator(task_id="start")
    
    # Task that might fail
    def task_that_might_fail(**context):
        """Task that may fail based on condition"""
        import random
        if random.random() < 0.5:
            raise Exception("Random failure occurred")
        return "Success"
    
    might_fail = PythonOperator(
        task_id="might_fail",
        python_callable=task_that_might_fail,
    )
    
    # Task that always succeeds
    always_succeed = EmptyOperator(task_id="always_succeed")
    
    # Task with trigger_rule="all_done" - runs regardless of upstream status
    cleanup_all_done = EmptyOperator(
        task_id="cleanup_all_done",
        trigger_rule="all_done",
    )
    
    # Task with trigger_rule="one_failed" - runs if any task failed
    handle_failure = EmptyOperator(
        task_id="handle_failure",
        trigger_rule="one_failed",
    )
    
    # Task with trigger_rule="none_failed_min_one_success" - runs if no failed and has success
    final_task = EmptyOperator(
        task_id="final_task",
        trigger_rule="none_failed_min_one_success",
    )
    
    end = EmptyOperator(
        task_id="end",
        trigger_rule="all_done",  # Always runs
    )
    
    # Define dependencies
    start >> [might_fail, always_succeed]
    [might_fail, always_succeed] >> cleanup_all_done
    might_fail >> handle_failure
    [cleanup_all_done, handle_failure] >> final_task >> end

# Create DAG
trigger_rules_dag_instance = trigger_rules_dag()

print("✅ Trigger Rules DAG created!")
print(f"Tasks: {[task.task_id for task in trigger_rules_dag_instance.tasks]}")
print("\n📊 Trigger Rules:")
print("  - cleanup_all_done: all_done (runs regardless of status)")
print("  - handle_failure: one_failed (runs if there's a failure)")
print("  - final_task: none_failed_min_one_success (runs if no failed and has success)")
print("  - end: all_done (always runs)")


## 5. Dynamic Task Mapping - Creating Dynamic Tasks

Dynamic Task Mapping allows creating multiple task instances from one task definition based on input data. Very useful for:
- Processing multiple files
- Batch operations
- Parallel processing


In [ ]:
# DAG with Dynamic Task Mapping
@dag(
    dag_id="dynamic_task_mapping_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["dynamic", "mapping"],
)
def dynamic_mapping_dag():
    """
    ### Dynamic Task Mapping Example
    Sample DAG using dynamic task mapping to create dynamic tasks.
    """
    
    # Task that creates list of items to process
    @task
    def get_items_to_process():
        """Return list of items to process"""
        items = [
            {"id": 1, "name": "item_1", "value": 100},
            {"id": 2, "name": "item_2", "value": 200},
            {"id": 3, "name": "item_3", "value": 300},
        ]
        print(f"Generated {len(items)} items to process")
        return items
    
    # Task with dynamic mapping - will create multiple task instances
    @task
    def process_item(item: dict):
        """Process one item - will be mapped for each item"""
        print(f"Processing {item['name']} with value {item['value']}")
        result = item['value'] * 2
        print(f"Result: {result}")
        return {"item_id": item['id'], "result": result}
    
    # Task to aggregate results
    @task
    def aggregate_results(results: list):
        """Aggregate results from all tasks"""
        total = sum(r['result'] for r in results)
        print(f"Aggregated total: {total}")
        return total
    
    # Define workflow with dynamic mapping
    items = get_items_to_process()
    # .expand() creates dynamic task instances for each item
    processed_items = process_item.expand(item=items)
    aggregate_results(processed_items)

# Create DAG
dynamic_mapping_dag_instance = dynamic_mapping_dag()

print("✅ Dynamic Task Mapping DAG created!")
print("\n💡 Dynamic mapping sẽ tạo:")
print("  - 1 task instance for get_items_to_process")
print("  - 3 task instances for process_item (one for each item)")
print("  - 1 task instance for aggregate_results")


## 6. TaskGroups - Organizing Tasks

TaskGroups allow grouping tasks together to:
- Improve visualization in UI
- Better organize code
- Apply settings to groups of tasks
- Create sub-workflows


In [ ]:
# DAG with TaskGroups
@dag(
    dag_id="taskgroup_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["taskgroup", "organization"],
)
def taskgroup_dag():
    """
    ### TaskGroup Example
    Sample DAG using TaskGroups to organize tasks.
    """
    
    start = EmptyOperator(task_id="start")
    
    # TaskGroup 1: Data Extraction
    with TaskGroup("extract_group") as extract_group:
        extract_a = EmptyOperator(task_id="extract_source_a")
        extract_b = EmptyOperator(task_id="extract_source_b")
        extract_c = EmptyOperator(task_id="extract_source_c")
        
        # Dependencies within group
        extract_a >> [extract_b, extract_c]
    
    # TaskGroup 2: Data Transformation
    with TaskGroup("transform_group") as transform_group:
        transform_a = EmptyOperator(task_id="transform_a")
        transform_b = EmptyOperator(task_id="transform_b")
        
        transform_a >> transform_b
    
    # TaskGroup 3: Data Loading
    with TaskGroup("load_group") as load_group:
        load_a = EmptyOperator(task_id="load_destination_a")
        load_b = EmptyOperator(task_id="load_destination_b")
        
        # Parallel loading
        [load_a, load_b]
    
    end = EmptyOperator(task_id="end")
    
    # Define dependencies between groups
    start >> extract_group >> transform_group >> load_group >> end

# Create DAG
taskgroup_dag_instance = taskgroup_dag()

print("✅ TaskGroup DAG created!")
print(f"Tasks: {[task.task_id for task in taskgroup_dag_instance.tasks]}")
print("\n📊 Task Groups:")
print("  - extract_group: extract_source_a → [extract_source_b, extract_source_c]")
print("  - transform_group: transform_a → transform_b")
print("  - load_group: [load_destination_a, load_destination_b] (parallel)")


## 7. Complex Branching Scenario - Real-world Example

Create a complex workflow with multiple branches and conditions to demonstrate how to use branching in practice.


In [ ]:
# Complex Branching Scenario
@dag(
    dag_id="complex_branching_scenario",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["complex", "branching", "real-world"],
)
def complex_branching_dag():
    """
    ### Complex Branching Scenario
    Sample DAG with complex branching logic for real-world use case.
    """
    
    start = EmptyOperator(task_id="start")
    
    # Check data quality
    def check_data_quality(**context):
        """Check data quality"""
        # Simulate data quality check
        import random
        quality_score = random.uniform(0.7, 1.0)
        
        if quality_score >= 0.9:
            return "high_quality"
        elif quality_score >= 0.8:
            return "medium_quality"
        else:
            return "low_quality"
    
    quality_check = BranchPythonOperator(
        task_id="check_data_quality",
        python_callable=check_data_quality,
    )
    
    # Different processing paths based on quality
    high_quality_process = EmptyOperator(task_id="high_quality_process")
    medium_quality_process = EmptyOperator(task_id="medium_quality_process")
    low_quality_process = EmptyOperator(task_id="low_quality_process")
    
    # Data cleaning for low quality
    clean_data = EmptyOperator(task_id="clean_data")
    low_quality_process >> clean_data
    
    # Join after processing
    join_processing = EmptyOperator(
        task_id="join_processing",
        trigger_rule="none_failed_min_one_success",
    )
    
    # Final validation
    validate_output = EmptyOperator(task_id="validate_output")
    
    # Notification based on result
    def send_notification(**context):
        """Send notification based on result"""
        ti = context['ti']
        # Check if validation passed
        print("Sending notification...")
        return "notification_sent"
    
    send_notif = PythonOperator(
        task_id="send_notification",
        python_callable=send_notification,
    )
    
    end = EmptyOperator(
        task_id="end",
        trigger_rule="all_done",
    )
    
    # Define dependencies
    start >> quality_check
    quality_check >> high_quality_process >> join_processing
    quality_check >> medium_quality_process >> join_processing
    quality_check >> low_quality_process
    clean_data >> join_processing
    join_processing >> validate_output >> send_notif >> end

# Create DAG
complex_branching_dag_instance = complex_branching_dag()

print("✅ Complex Branching DAG created!")
print(f"Tasks: {[task.task_id for task in complex_branching_dag_instance.tasks]}")
print("\n📊 Workflow:")
print("  start → check_data_quality → [high/medium/low_quality_process]")
print("  low_quality_process → clean_data → join_processing")
print("  join_processing → validate_output → send_notification → end")


## 9. Summary and Next Steps

### ✅ What we learned:
1. Bitshift operators (>>, <<) - Define dependencies
2. BranchPythonOperator - Conditional branching
3. Trigger rules - Handle failures and skipped tasks
4. Dynamic Task Mapping - Create dynamic tasks
5. TaskGroups - Organize tasks
6. Complex branching scenarios - Real-world use cases
7. Best practices and common pitfalls

### 📚 Next Lab:
- **Lab 5**: XCom and Data Sharing
- XCom push/pull
- Task return values
- Custom XCom backends
- Data passing best practices

### 🔗 Useful Links:
- [Task Dependencies](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dags.html#task-dependencies)
- [Branching](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dags.html#branching)
- [Trigger Rules](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/tasks.html#trigger-rules)
- [Dynamic Task Mapping](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dynamic-task-mapping.html)
- [TaskGroups](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/taskgroup.html)

### 💡 Exercises:
1. Create DAG with branching based on data quality
2. Use dynamic mapping to process multiple files
3. Create TaskGroups for ETL pipeline
4. Implement error handling with trigger rules
5. Build complex workflow with multiple branches
